# Xóa hết bình luận rỗng

In [1]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_json('total_comment(13-4-2022).json')
print(df.head())
df.info()

In [ ]:
# bình luận theo số sao
r0 = df[df['rate'] == 0]
r1 = df[df['rate'] == 1]
r2 = df[df['rate'] == 2]
r3 = df[df['rate'] == 3]
r4 = df[df['rate'] == 4]
r5 = df[df['rate'] == 5]

In [ ]:
# Bình luận rỗng theo số sao
r0_empty = r0[r0['content'] == '']
r1_empty = r1[r1['content'] == '']
r2_empty = r2[r2['content'] == '']
r3_empty = r3[r3['content'] == '']
r4_empty = r4[r4['content'] == '']
r5_empty = r5[r5['content'] == '']

In [ ]:
# Tổng bình luận rỗng
print('số bl rỗng 0s',len(r0_empty))
print('số bl rỗng 1s',len(r1_empty))
print('số bl rỗng 2s',len(r2_empty))
print('số bl rỗng 3s',len(r3_empty))
print('số bl rỗng 4s',len(r4_empty))
print('số bl rỗng 5s',len(r5_empty))

In [ ]:
# Xóa tất cả bl rỗng
df = df.drop(r0_empty.index)
df = df.drop(r1_empty.index)
df = df.drop(r2_empty.index)
df = df.drop(r3_empty.index)
df = df.drop(r4_empty.index)
df = df.drop(r5_empty.index)

# NLP processing


In [2]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import os
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import re
from unicodedata import normalize
import requests
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import KeyedVectors
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.metrics import classification_report
import numpy


In [ ]:
X1 = df['content']
y = df['rate']

In [ ]:
# Chuẩn hóa: thay thế từ viết tắt và xóa các từ dừng 
td = open('tu_dung', 'r', encoding='UTF-8').readlines()
vt = open('viet_tat', 'r', encoding='UTF-8').readlines()

In [ ]:
# Chuyển sang viết thường
for e in X1:
    e.lower()

In [ ]:
# Thay thế từ viết tắt
def dong_nghia(self):
    list_text = self.split(' ')
    for i in range(len(list_text)):
        for j in range(len(vt)):
            if(list_text[i] == vt[j][0]):
                list_text[i] = vt[j][1]
    self = ' '.join(list_text)
    return self

In [ ]:
def tu_dung(self):
    words = []
    split_words = self.split()
    for word in split_words:
        if word not in td:
            words.append(word)
    return ' '.join(words) 

In [ ]:
# Xóa từ dừng 
X2 = []
for e in X1:
    X2.append(tu_dung(e))
    

In [ ]:
X3 = []
for e in X2:
    X3.append(dong_nghia(e))

In [ ]:
# Xóa ký tự đặc biệt và tách từ
X = []
for e in X3:
    lines = gensim.utils.simple_preprocess(e)
    lines = ' '.join(lines)
    lines = ViTokenizer.tokenize(lines)
    X.append(lines)


In [ ]:
import pickle
# File pickle chứa cmt đã đc xóa ký tự đb và tt
# pickle.dump(X, open('X_data.pkl', 'wb'))
# pickle.dump(y, open('y_data.pkl', 'wb'))

In [3]:
X = pd.read_pickle(r'X_data.pkl')
y = pd.read_pickle(r'y_data.pkl')

In [7]:
y[11]

1

In [ ]:
# chia nhãn
for index, item in enumerate(y):
    switch(item){
	if item < 3:
		y[index] = 'neg'
    elif item > 3:
        y[index] = 'pos'
    elif item == 3:
        y[index] = 'neu'
print(y[1])

In [8]:
# 0 là tiêu cực 1 là tích cực
arr = numpy.asarray(y)
arr[ arr == 1 ] = 0
arr[ arr == 2 ] = 0
arr[ arr == 3 ] = 1
arr[ arr == 4 ] = 1
arr[ arr == 5 ] = 1
print(arr)

[0 0 1 ... 1 0 0]


In [10]:
#pickle.dump(y, open('y.pkl', 'wb'))
y = pd.read_pickle(r'y.pkl')

In [13]:
# Chia tập dữ liệu để kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
# pickle.dump(X, open('X_train_count.pkl', 'wb'))
# pickle.dump(y, open('X_test_count.pkl', 'wb'))

# Xây dựng mô hình

Phân loại văn bản với Naive Bayes

In [14]:
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [15]:
start_time = time.time()
# An n-gram is just a string of n words in a row
# max_df loại những từ xuất hiện >80% tdl
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')


Done training Naive Bayes in 36.631016969680786 seconds.


In [16]:
# Save model
pickle.dump(text_clf, open('model/naive_bayes.pkl', 'wb'))

In [17]:
# Naive Bayes
nb_model = pd.read_pickle('model/naive_bayes.pkl')
y_pred = nb_model.predict(X_test)
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))

Naive Bayes, Accuracy = 0.9214670418755474


Phân loại văn bản với LogisticRegression

In [18]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs',
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open('model/linear_classifier.pkl', 'wb'))

Done training Linear Classifier in 183.32998919487 seconds.


In [19]:
lr_model = pd.read_pickle('model/linear_classifier.pkl')
y_pred = lr_model.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))

SVM, Accuracy = 0.9400405591538487


Phân loại văn bản với PhoBert

# Đánh giá mô hình phân loại cho văn bản

In [76]:
len(y_test)
y_test = y_test.tolist()

In [79]:
# Test thử mô hình naive bayes với tập test
for i in range(0, 20):
    test = nb_model.predict([X_test[i]])
    print(test)
    print(y_test[i])
    print('---')

[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
0
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---


In [86]:
test = 'không thích sản phẩm'
test = nb_model.predict([X_test[i]])
print(test)
test = 'thích sản phẩm'
test = nb_model.predict([X_test[i]])
print(test)

[1]
[1]


In [81]:
# Test thử mô hình hồi quy với tập test
for i in range(20, 40):
    test = lr_model.predict([X_test[i]])
    print(test)
    print(y_test[i])
    print('---')

[1]
1
---
[1]
1
---
[1]
1
---
[1]
0
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[0]
0
---
[1]
1
---
[1]
1
---
[1]
1
---
[1]
1
---
[0]
0
---
[1]
1
---


In [85]:
test = 'không thích sản phẩm'
test = lr_model.predict([X_test[i]])
print(test)
test = 'thích sản phẩm'
print(test)

[1]
[1]


In [20]:
from sklearn.metrics import classification_report

# Naive_bayes

nb_model = pd.read_pickle('model/naive_bayes.pkl')
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.81      0.26      0.39     57709
           1       0.93      0.99      0.96    532540

    accuracy                           0.92    590249
   macro avg       0.87      0.63      0.68    590249
weighted avg       0.91      0.92      0.90    590249



In [60]:
test = [X_test[211]]
test = lr_model.predict(test)
print(test)
X_test[211]

[0]


'cáp xuất hình thất_vọng'

In [22]:
# linear_classifier

lr_model = pd.read_pickle('model/linear_classifier.pkl')
y_pred = lr_model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.76      0.57      0.65     57709
           1       0.95      0.98      0.97    532540

    accuracy                           0.94    590249
   macro avg       0.86      0.78      0.81    590249
weighted avg       0.94      0.94      0.94    590249

